In [12]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tempor/pred.csv
/kaggle/input/rssssss/Predictions.csv
/kaggle/input/hatespeech/combined_file.csv
/kaggle/input/hatespeech/train_data_70.csv
/kaggle/input/hatespeech/train_80_data.csv
/kaggle/input/hatespeech/unseen_test.csv
/kaggle/input/hatespeech/train-final.csv
/kaggle/input/hatespeech/train_data_75.csv
/kaggle/input/hatespeech/test-final.csv
/kaggle/input/hatespeech/test_20_data.csv
/kaggle/input/hatespeech/test_data_30.csv
/kaggle/input/hatespeech/test_data_25.csv


In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from scipy.sparse import hstack
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
from textblob import TextBlob
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import string
from textblob import TextBlob

In [14]:
# Download NLTK resources (if not downloaded)
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [15]:
# Load training and testing datasets
train_data = pd.read_csv('/kaggle/input/hatespeech/train_80_data.csv')  # Replace 'training_data.csv' with your training dataset file
test_data = pd.read_csv('/kaggle/input/hatespeech/test_20_data.csv')    # Replace 'testing_data.csv' with your testing dataset file


In [16]:
# Preprocessing text data
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

In [17]:
# Extended text preprocessing function
def preprocess_text_extended(text):
    # Remove punctuation and symbols
    text = re.sub(r'[^\w\s]', '', text)
    
    # Tokenization
    tokens = word_tokenize(text)
    
    # Remove stopwords and normalize
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word.lower() for word in tokens if word.lower() not in stop_words]
    
    # Sentiment analysis
    sentiment = TextBlob(text).sentiment.polarity
    
    # Stemming
    stemmer = PorterStemmer()
    stemmed_tokens = [stemmer.stem(word) for word in filtered_tokens]
    
    return ' '.join(stemmed_tokens)

train_data['processed_text'] = train_data['text'].apply(preprocess_text_extended)
train_data['processed_context'] = train_data['context'].apply(preprocess_text_extended)

test_data['processed_text'] = test_data['text'].apply(preprocess_text_extended)
test_data['processed_context'] = test_data['context'].apply(preprocess_text_extended)

# Feature extraction using TF-IDF separately for text and context in training data
tfidf_vectorizer_text = TfidfVectorizer(max_features=1000)  # You can adjust max_features as needed
X_train_text = tfidf_vectorizer_text.fit_transform(train_data['processed_text'].astype(str))

tfidf_vectorizer_context = TfidfVectorizer(max_features=1000)  # You can adjust max_features as needed
X_train_context = tfidf_vectorizer_context.fit_transform(train_data['processed_context'].astype(str))

# Feature extraction using TF-IDF separately for text and context in testing data
X_test_text = tfidf_vectorizer_text.transform(test_data['processed_text'].astype(str))
X_test_context = tfidf_vectorizer_context.transform(test_data['processed_context'].astype(str))



In [18]:
X_train_combined= X_train_text
X_test_combined = X_test_text
y_train = train_data['label']
y_test = test_data['label']

In [19]:
from sklearn.naive_bayes import MultinomialNB

# Training the Naive Bayes model
nb_model = MultinomialNB()
nb_model.fit(X_train_combined, y_train)

# Predictions on the test set using Naive Bayes
y_pred_nb = nb_model.predict(X_test_combined)

# Model evaluation for Naive Bayes
accuracy_nb = accuracy_score(y_test, y_pred_nb)
print(f"Accuracy (Naive Bayes): {accuracy_nb:.2f}")

# Additional evaluation metrics for Naive Bayes (optional)
print("Classification Report (Naive Bayes):")
print(classification_report(y_test, y_pred_nb))


Accuracy (Naive Bayes): 0.67
Classification Report (Naive Bayes):
              precision    recall  f1-score   support

           0       0.68      0.90      0.77      1187
           1       0.64      0.29      0.40       713

    accuracy                           0.67      1900
   macro avg       0.66      0.60      0.59      1900
weighted avg       0.66      0.67      0.64      1900



In [20]:


# Calculate Accuracy
accuracy = accuracy_score(y_test, y_pred_nb)
print(f"Accuracy: {accuracy:.2f}")

# Calculate Precision
precision = precision_score(y_test, y_pred_nb, average='weighted')
print(f"Precision: {precision:.2f}")

# Calculate Recall
recall = recall_score(y_test, y_pred_nb, average='weighted')
print(f"Recall: {recall:.2f}")

# Calculate F1 Score
f1 = f1_score(y_test, y_pred_nb, average='weighted')
print(f"F1 Score: {f1:.2f}")

# Classification Report (Includes Precision, Recall, F1-score)
print(classification_report(y_test, y_pred_nb))

Accuracy: 0.67
Precision: 0.66
Recall: 0.67
F1 Score: 0.64
              precision    recall  f1-score   support

           0       0.68      0.90      0.77      1187
           1       0.64      0.29      0.40       713

    accuracy                           0.67      1900
   macro avg       0.66      0.60      0.59      1900
weighted avg       0.66      0.67      0.64      1900



In [21]:
name = 'NB 80-20'
test_data[name] = y_pred_nb
test_data.to_csv(name+'.csv',index=False)

In [22]:
# pred.to_csv('ML_Predictions.csv',index=False)